In [5]:
import pandas as pd
import numpy as np
import seaborn as sns

from scipy.stats import mstats
import statsmodels.api as sm

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression, Lasso, LassoCV
from sklearn.metrics import recall_score,accuracy_score, mean_squared_error, r2_score, log_loss, precision_score

from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler

from sklearn.tree import DecisionTreeClassifier

import matplotlib.pyplot as plt
%matplotlib inline

from scipy.stats import ttest_ind

from statsmodels.stats.outliers_influence import variance_inflation_factor

plt.rcParams['font.family'] = 'Malgun Gothic'

import warnings
warnings.filterwarnings('ignore')


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score,accuracy_score, precision_score,roc_auc_score,f1_score,confusion_matrix
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from pytorch_tabnet.tab_model import TabNetClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC


## 0.파일 불러오기

In [7]:
df = pd.read_csv('../data/processed_data/재무비율_재무지표_concat.csv', index_col=0)
df

,회사명,거래소코드,회계년도,소속코드_df1,통계청 한국표준산업분류 코드 10차(대분류)_df1,산업코드_df1,산업명_df1,상장일_df1,상장폐지일_df1,현금및현금성자산(IFRS),...,자본분배율(IFRS),이윤분배율(IFRS),법인세비용차감전(계속사업)손익(IFRS)(백만원),인건비(IFRS)(백만원),금융비용(IFRS)(백만원),임차료(IFRS)(백만원),세금과공과(IFRS)(백만원),감가상각비(IFRS)(백만원),종업원수(IFRS),label_df2
0,(주)다이나믹디자인,145210,2015,1,29,32902,기타 기계 및 장비 제조업,2015-03-19,0,10376890.0,...,29.18,2.06,1548.77,46876.84,3589.40,0.0,0.0,14203.75,150.0,1
1,(주)다이나믹디자인,145210,2016,1,29,32902,기타 기계 및 장비 제조업,2015-03-19,0,15192647.0,...,0.00,0.00,-11379.61,0.00,0.00,0.0,0.0,0.00,134.0,1
2,(주)다이나믹디자인,145210,2017,1,29,32902,기타 기계 및 장비 제조업,2015-03-19,0,6564156.0,...,-90.09,-161.93,-34429.26,39598.55,3990.88,0.0,0.0,11694.91,526.0,1
3,(주)대우건설,47040,2012,1,41,64101,종합 건설업,2001-03-23,0,388440274.0,...,40.72,12.78,227947.72,802635.00,94086.89,124047.0,54145.0,51212.00,5192.0,0
4,(주)대우건설,47040,2013,1,41,64101,종합 건설업,2001-03-23,0,409596263.0,...,-125.45,-177.13,-839724.66,913926.00,86015.38,123239.0,62921.0,59003.00,6382.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
384,한솔아이원스(주),114810,2017,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,0,9892897.0,...,40.36,-1.99,-79.97,19022.41,5048.10,0.0,0.0,7923.59,519.0,1
385,한솔아이원스(주),114810,2018,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,0,5510953.0,...,32.63,2.39,-678.20,24417.40,3831.05,0.0,0.0,8695.36,608.0,1
386,한솔아이원스(주),114810,2019,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,0,12310161.0,...,22.96,-2.27,-2987.27,30291.89,3425.26,0.0,0.0,8607.60,538.0,1
387,한솔아이원스(주),114810,2020,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,0,9274990.0,...,36.33,8.28,6268.86,28696.72,2408.62,0.0,0.0,7715.50,539.0,1


In [8]:
df.columns

Index(['회사명', '거래소코드', '회계년도', '소속코드_df1', '통계청 한국표준산업분류 코드 10차(대분류)_df1',
       '산업코드_df1', '산업명_df1', '상장일_df1', '상장폐지일_df1', '현금및현금성자산(IFRS)',
       ...
       '자본분배율(IFRS)', '이윤분배율(IFRS)', '법인세비용차감전(계속사업)손익(IFRS)(백만원)',
       '인건비(IFRS)(백만원)', '금융비용(IFRS)(백만원)', '임차료(IFRS)(백만원)',
       '세금과공과(IFRS)(백만원)', '감가상각비(IFRS)(백만원)', '종업원수(IFRS)', 'label_df2'],
      dtype='object', length=204)

In [9]:
X_data = df[df.columns[9:-1]]
y_data = df['label_df2']

## 1.데이터 split

In [10]:
X_train,X_test,y_train,y_test = train_test_split(X_data,y_data,test_size=0.3,stratify=y_data,random_state = 40)

In [11]:
n, d = X_train.shape
print("number of feature:", d)
print("number of feature:", n)

number of feature: 194
number of feature: 272


In [12]:
X_train.describe()

,현금및현금성자산(IFRS),유동금융자산(IFRS),매출채권 및 기타유동채권(IFRS),상각후원가측정 유가증권(IFRS),매출채권(IFRS),미수금(IFRS),단기대여금(IFRS),선급금(IFRS),기타유동자산(IFRS),상품(IFRS),...,노동소득분배율(IFRS),자본분배율(IFRS),이윤분배율(IFRS),법인세비용차감전(계속사업)손익(IFRS)(백만원),인건비(IFRS)(백만원),금융비용(IFRS)(백만원),임차료(IFRS)(백만원),세금과공과(IFRS)(백만원),감가상각비(IFRS)(백만원),종업원수(IFRS)
count,2.720000e+02,2.720000e+02,2.720000e+02,2.720000e+02,2.720000e+02,2.720000e+02,2.720000e+02,2.720000e+02,2.720000e+02,2.720000e+02,...,272.000000,272.000000,272.000000,2.720000e+02,2.720000e+02,272.000000,272.000000,272.000000,272.000000,272.000000
mean,8.490363e+07,6.375838e+07,2.710998e+08,7.707054e+04,1.252311e+08,3.269555e+07,1.209314e+07,4.047387e+07,3.512398e+07,6.673554e+06,...,199.270588,-116.182353,-308.894228,3.241004e+04,9.685671e+04,8036.948493,6564.474485,884.555588,22376.288603,711.356618
std,3.059550e+08,2.430738e+08,1.014358e+09,1.230775e+06,4.328492e+08,1.349381e+08,3.937337e+07,1.644082e+08,1.474732e+08,2.435779e+07,...,1762.905771,1761.386444,3267.589462,2.292609e+05,2.897539e+05,43696.153379,51109.546719,5532.381911,65851.384635,1678.825189
min,8.790000e+02,0.000000e+00,2.830360e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000,-28477.690000,-50846.420000,-1.010005e+06,4.737700e+02,-54363.000000,0.000000,0.000000,23.750000,0.000000
25%,2.274892e+06,1.095000e+06,9.645330e+06,0.000000e+00,8.344018e+06,3.731550e+05,2.000000e+03,3.997692e+05,2.528150e+05,0.000000e+00,...,34.545000,0.000000,-15.222500,-9.493108e+03,5.902865e+03,197.322500,0.000000,0.000000,1447.005000,83.750000
50%,7.973951e+06,4.014943e+06,2.187085e+07,0.000000e+00,1.875475e+07,1.535465e+06,7.250980e+05,1.929540e+06,9.571845e+05,4.102255e+05,...,57.710000,29.385000,1.910000,8.594100e+02,1.416852e+04,1258.100000,0.000000,0.000000,3011.750000,195.500000
75%,3.284412e+07,1.869723e+07,6.865800e+07,0.000000e+00,4.331008e+07,7.408619e+06,5.604768e+06,7.368950e+06,4.551894e+06,2.700278e+06,...,76.590000,50.855000,21.325000,1.094945e+04,3.845116e+04,3276.457500,234.635000,0.000000,9221.575000,380.500000
max,2.542381e+09,2.099634e+09,7.980449e+09,2.029100e+07,3.595024e+09,1.095078e+09,2.832180e+08,1.743728e+09,1.303862e+09,2.623750e+08,...,28577.690000,86.760000,77.600000,1.571030e+06,2.055868e+06,459499.000000,602155.270000,62921.000000,513111.000000,13602.000000


In [13]:
y_test.value_counts()

label_df2
1    89
0    28
Name: count, dtype: int64

## 6. 모델링

### 6.1 로지스틱 회귀 분석

In [14]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, 
    roc_curve, auc, RocCurveDisplay
)
import matplotlib.pyplot as plt


## 6-2.catboost

In [15]:
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# CatBoost 모델 생성
model = CatBoostClassifier(iterations=100, learning_rate=0.1, loss_function='Logloss')

# 모델 학습
model.fit(X_train, y_train, verbose=0)

# 테스트 데이터에 대한 예측
y_pred2 = model.predict(X_test)

# 다양한 평가 지표 출력
accuracy = accuracy_score(y_test, y_pred2)
precision = precision_score(y_test, y_pred2)
recall = recall_score(y_test, y_pred2)
f1 = f1_score(y_test, y_pred2)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Accuracy: 0.8290598290598291
Precision: 0.8349514563106796
Recall: 0.9662921348314607
F1 Score: 0.8958333333333334


### 6-3.Decision Tree

In [16]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Decision Tree 모델 생성
model = DecisionTreeClassifier()

# 모델 학습
model.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
y_pred = model.predict(X_test)

# 다양한 평가 지표 출력
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Accuracy: 0.7521367521367521
Precision: 0.8125
Recall: 0.8764044943820225
F1 Score: 0.8432432432432433


### 6-4.나이브베이즈

In [17]:
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# 기본 나이브 베이즈 분류기 생성
base_model = GaussianNB()

# 배깅 분류기 생성
model = BaggingClassifier(base_model, n_estimators=10, random_state=42)

# 모델 학습
model.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
y_pred = model.predict(X_test)

# 다양한 평가 지표 출력
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.38461538461538464
Precision: 0.9047619047619048
Recall: 0.21348314606741572
F1 Score: 0.34545454545454546


In [18]:
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

In [19]:
df2 = pd.DataFrame()

In [20]:
logit = LogisticRegression()
logit.fit(X_train, y_train)
y_pred_train = logit.predict(X_train)
y_pred_test = logit.predict(X_test)

log_train = [accuracy_score(y_train, y_pred_train),precision_score(y_train, y_pred_train),recall_score(y_train, y_pred_train),f1_score(y_train, y_pred_train),roc_auc_score(y_train, y_pred_train)]
log_test = [accuracy_score(y_test, y_pred_test),precision_score(y_test, y_pred_test),recall_score(y_test, y_pred_test),f1_score(y_test, y_pred_test),roc_auc_score(y_test, y_pred_test)]

df2['logit_train'] = log_train
df2['logit_test'] = log_test

print(confusion_matrix(y_test,y_pred_test))

[[15 13]
 [14 75]]


In [21]:
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
y_pred_train = rf.predict(X_train)
y_pred_test = rf.predict(X_test)

rf_train = [accuracy_score(y_train, y_pred_train),precision_score(y_train, y_pred_train),recall_score(y_train, y_pred_train),f1_score(y_train, y_pred_train),roc_auc_score(y_train, y_pred_train)]
rf_test = [accuracy_score(y_test, y_pred_test),precision_score(y_test, y_pred_test),recall_score(y_test, y_pred_test),f1_score(y_test, y_pred_test),roc_auc_score(y_test, y_pred_test)]

df2['rf_train'] = rf_train
df2['rf_test'] = rf_test


print(confusion_matrix(y_test,y_pred_test))

[[12 16]
 [ 3 86]]


In [22]:
xgb = XGBClassifier(random_state=42)
xgb.fit(X_train, y_train)
y_pred_train = xgb.predict(X_train)
y_pred_test = xgb.predict(X_test)

xgb_train = [accuracy_score(y_train, y_pred_train),precision_score(y_train, y_pred_train),recall_score(y_train, y_pred_train),f1_score(y_train, y_pred_train),roc_auc_score(y_train, y_pred_train)]
xgb_test = [accuracy_score(y_test, y_pred_test),precision_score(y_test, y_pred_test),recall_score(y_test, y_pred_test),f1_score(y_test, y_pred_test),roc_auc_score(y_test, y_pred_test)]

df2['xgb_train'] = xgb_train
df2['xgb_test'] = xgb_test


print(confusion_matrix(y_test,y_pred_test))

[[10 18]
 [ 3 86]]


In [23]:
lgb = LGBMClassifier(random_state=42)
lgb.fit(X_train, y_train)
y_pred_train = lgb.predict(X_train)
y_pred_test = lgb.predict(X_test)

lgb_train =[accuracy_score(y_train, y_pred_train),precision_score(y_train, y_pred_train),recall_score(y_train, y_pred_train),f1_score(y_train, y_pred_train),roc_auc_score(y_train, y_pred_train)]
lgb_test = [accuracy_score(y_test, y_pred_test),precision_score(y_test, y_pred_test),recall_score(y_test, y_pred_test),f1_score(y_test, y_pred_test),roc_auc_score(y_test, y_pred_test)]

df2['lgb_train'] = lgb_train
df2['lgb_test'] = lgb_test


print(confusion_matrix(y_test,y_pred_test))

LightGBMError: Do not support special JSON characters in feature name.

In [24]:
cat = CatBoostClassifier(random_state=42, verbose=0)
cat.fit(X_train, y_train)
y_pred_train = cat.predict(X_train)
y_pred_test = cat.predict(X_test)

cat_train = [accuracy_score(y_train, y_pred_train),precision_score(y_train, y_pred_train),recall_score(y_train, y_pred_train),f1_score(y_train, y_pred_train),roc_auc_score(y_train, y_pred_train)]
cat_test = [accuracy_score(y_test, y_pred_test),precision_score(y_test, y_pred_test),recall_score(y_test, y_pred_test),f1_score(y_test, y_pred_test),roc_auc_score(y_test, y_pred_test)]

df2['cat_train'] = cat_train
df2['cat_test'] = cat_test


print(confusion_matrix(y_test,y_pred_test))

[[12 16]
 [ 1 88]]


In [25]:
svc_clf = SVC() 
svc_clf.fit(X_train,y_train)
y_pred_train = svc_clf.predict(X_train)
y_pred_test = svc_clf.predict(X_test)

svc_clf_train =[accuracy_score(y_train, y_pred_train),precision_score(y_train, y_pred_train),recall_score(y_train, y_pred_train),f1_score(y_train, y_pred_train),roc_auc_score(y_train, y_pred_train)]
svc_clf_test = [accuracy_score(y_test, y_pred_test),precision_score(y_test, y_pred_test),recall_score(y_test, y_pred_test),f1_score(y_test, y_pred_test),roc_auc_score(y_test, y_pred_test)]

df2['svm_train'] = svc_clf_train
df2['svm_test'] = svc_clf_test


print(confusion_matrix(y_test,y_pred_test))

[[ 1 27]
 [ 0 89]]


In [27]:
# clf = TabNetClassifier()

# # 모델 훈련
# clf.fit(
#     X_train.values, y_train.values,
#     eval_set=[(X_train.values, y_train.values), (X_test.values, y_test.values)],
#     eval_name=['train', 'test'],
#     eval_metric=['accuracy'],
#     max_epochs=10,
#     patience=10,
#     batch_size=1024, 
#     virtual_batch_size=128,
#     num_workers=0,
#     drop_last=False
# )

In [28]:
# y_pred_train = clf.predict(X_train.values)
# y_pred_test = clf.predict(X_test.values)

# tab_train = [accuracy_score(y_train, y_pred_train),precision_score(y_train, y_pred_train),recall_score(y_train, y_pred_train),f1_score(y_train, y_pred_train),roc_auc_score(y_train, y_pred_train)]
# tab_test = [accuracy_score(y_test, y_pred_test),precision_score(y_test, y_pred_test),recall_score(y_test, y_pred_test),f1_score(y_test, y_pred_test),roc_auc_score(y_test, y_pred_test)]

# df['tabnet_train'] = tab_train
# df['tabnet_test'] = tab_test


# print(confusion_matrix(y_test,y_pred_test))

In [29]:
# import torch
# import pytorch_tabnet

# print("PyTorch Version:", torch.__version__)
# print("pytorch_tabnet Version:", pytorch_tabnet.__version__)


In [26]:
df2.index = ['accuracy','precision','recall','f1-score','roc-auc']
df2

,logit_train,logit_test,rf_train,rf_test,xgb_train,xgb_test,cat_train,cat_test,svm_train,svm_test
accuracy,0.875000,0.769231,1.0,0.837607,1.0,0.820513,1.0,0.854701,0.775735,0.769231
precision,0.891403,0.852273,1.0,0.843137,1.0,0.826923,1.0,0.846154,0.772388,0.767241
recall,0.951691,0.842697,1.0,0.966292,1.0,0.966292,1.0,0.988764,1.000000,1.000000
f1-score,0.920561,0.847458,1.0,0.900524,1.0,0.891192,1.0,0.911917,0.871579,0.868293
roc-auc,0.791230,0.689205,1.0,0.697432,1.0,0.661717,1.0,0.708668,0.530769,0.517857
